In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D

my_VGG16 = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), input_shape=(224, 224, 3), padding='same', activation='relu'))

# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

# Ajout de la première couche de pooling
my_VGG16.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

In [3]:
from keras.layers import Flatten, Dense

my_VGG16.add(Flatten())  # Conversion des matrices 3D en vecteur 1D

# Ajout de la première couche fully-connected, suivie d'une couche ReLU
my_VGG16.add(Dense(4096, activation='relu'))

# Ajout de la deuxième couche fully-connected, suivie d'une couche ReLU
my_VGG16.add(Dense(4096, activation='relu'))

# Ajout de la dernière couche fully-connected qui permet de classifier
my_VGG16.add(Dense(1000, activation='softmax'))

ResourceExhaustedError: OOM when allocating tensor with shape[802816,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

## Utilisation d'un VGG16 pre entrainé ##

In [4]:
from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

553476096/553467096 [==============================] - 141s 0us/step


In [6]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img('werewolf.jpeg', target_size=(224, 224))  # Charger l'image
img = img_to_array(img)  # Convertir en tableau numpy
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [7]:
y = model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)

In [8]:
from keras.applications.vgg16 import decode_predictions

# Afficher les 3 classes les plus probables
print('Top 3 :', decode_predictions(y, top=3)[0])

49152/35363 [=========================================] - 0s 1us/step
Top 3 : [('n01990800', 'isopod', 0.17689538), ('n02091134', 'whippet', 0.08789499), ('n02124075', 'Egyptian_cat', 0.06892992)]


## Transfert Learning ##

In [9]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras import Model
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Récupérer la sortie de ce réseau
x = model.output

# Ajouter la nouvelle couche fully-connected pour la classification à 10 classes
predictions = Dense(10, activation='softmax')(x)

# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

58900480/58889256 [==============================] - 15s 0us/step


### Strat 1 - Fine-tuning ###

On entraine tout le reseau, il faut donc rendre toutes les couches 'entrainables'. 

Utilisable si on a beaucoup d'images d'entrainement et un but different d'ImageNet.

In [10]:
for layer in model.layers:
   layer.trainable = True

### Strat 2 - Extraction de features ###

On entraine juste le nouveau classifieur et on ne re entraine pas les autres couches.

Utilisable si on a peu d'images d'entrainement et qu'elles sont similaires a celles d'ImageNet.

In [11]:
for layer in model.layers:
   layer.trainable = False

### Strat 3 - Fine-tuning Partiel ###

On entraine le nouveau classifieur et les couches hautes.

Utilisable quand le jeu d'images est petit et different du jeu d'entrainement. Compromis.

In [12]:
# Ne pas entraîner les 5 premières couches (les plus basses) 
for layer in model.layers[:5]:
   layer.trainable = False

### Entrainement du Reseau ###

Compilation du nouveau modele et entrainement.

In [13]:
# Compiler le modèle 
new_model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Entraîner sur les données d'entraînement (X_train, y_train)
model_info = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)


NameError: name 'optimizers' is not defined